## Camada Gold

- Aqui criarei as tabelas de visao mais analitica e agregada, de facil consumo para as areas de negocio

In [0]:
from pyspark.sql.functions import sum, count, coalesce, lit, countDistinct

In [0]:
fato_sales = spark.table("ab_inbev.silver.fato_sales")
dim_date = spark.table("ab_inbev.silver.dim_date")

df = fato_sales.join(dim_date, fato_sales.DATE == dim_date.DATE, "left") \
    .groupBy(dim_date.YEAR, dim_date.MONTH, dim_date.PERIOD) \
    .agg(sum(fato_sales.Volume).alias("total_volume"), count("*").alias("sales_count"))

display(df)

#escrita
df.write.mode("overwrite").saveAsTable("ab_inbev.gold.tb_volume_periodo")

In [0]:
dim_brand = spark.table("ab_inbev.silver.dim_brand")
dim_date = spark.table("ab_inbev.silver.dim_date")
fato_sales = spark.table("ab_inbev.silver.fato_sales")

#cross join para termos ate os registros de 0 vendas na tabela
df = dim_brand.crossJoin(dim_date) \
    .join(fato_sales, (fato_sales.DATE == dim_date.DATE) & (fato_sales.CE_BRAND_FLVR == dim_brand.CE_BRAND_FLVR), "left") \
    .groupBy(dim_brand.BRAND_NM, dim_date.YEAR, dim_date.MONTH) \
    .agg(coalesce(sum(fato_sales.Volume), lit(0)).alias("total_volume"), countDistinct(fato_sales.sale_id).alias("sales_count"))

display(df)

#escrita
df.write.mode("overwrite").saveAsTable("ab_inbev.gold.tb_volume_brand_mes")

In [0]:
dim_channels = spark.table("ab_inbev.silver.dim_channels")
dim_date = spark.table("ab_inbev.silver.dim_date")
fato_sales = spark.table("ab_inbev.silver.fato_sales")

df = dim_channels.crossJoin(dim_date) \
    .join(fato_sales, (fato_sales.DATE == dim_date.DATE) & (fato_sales.TRADE_CHNL_DESC == dim_channels.TRADE_CHNL_DESC), "left") \
    .groupBy(dim_channels.TRADE_GROUP_DESC, dim_date.YEAR, dim_date.MONTH) \
    .agg(coalesce(sum(fato_sales.Volume), lit(0)).alias("total_volume"), countDistinct(fato_sales.sale_id).alias("sales_count"))

display(df)

#escrita
df.write.mode("overwrite").saveAsTable("ab_inbev.gold.tb_volume_tradeGroup_mes")

In [0]:
dim_pkg = spark.table("ab_inbev.silver.dim_pkg")
dim_date = spark.table("ab_inbev.silver.dim_date")
fato_sales = spark.table("ab_inbev.silver.fato_sales")

df = dim_pkg.crossJoin(dim_date) \
    .join(fato_sales, (fato_sales.DATE == dim_date.DATE) & (fato_sales.TSR_PCKG_NM == dim_pkg.TSR_PCKG_NM), "left") \
    .groupBy(dim_pkg.PKG_CAT, dim_date.YEAR, dim_date.MONTH) \
    .agg(coalesce(sum(fato_sales.Volume), lit(0)).alias("total_volume"), countDistinct(fato_sales.sale_id).alias("sales_count"))

display(df)

#escrita
df.write.mode("overwrite").saveAsTable("ab_inbev.gold.tb_volume_pkg_mes")

In [0]:
df = fato_sales.groupBy("Btlr_Org_LVL_C_Desc") \
    .agg(sum("Volume").alias("total_volume"), count("*").alias("sales_count"))

display(df)

#escrita
df.write.mode("overwrite").saveAsTable("ab_inbev.gold.tb_volume_regiao")

In [0]:
fato_sales = spark.table("ab_inbev.silver.fato_sales")
dim_date = spark.table("ab_inbev.silver.dim_date")

df = fato_sales.join(dim_date, fato_sales.DATE == dim_date.DATE, "left") \
    .groupBy(fato_sales.Btlr_Org_LVL_C_Desc, dim_date.YEAR, dim_date.MONTH) \
    .agg(sum(fato_sales.Volume).alias("total_volume"), count("*").alias("sales_count"))

display(df)

#escrita
df.write.mode("overwrite").saveAsTable("ab_inbev.gold.tb_volume_regiao_mes")

In [0]:
#me arrependi de nao ter feito uma dim de regiao
df = spark.sql(""" 
    with dim_reg as (
    select Btlr_Org_LVL_C_Desc
    from ab_inbev.silver.fato_sales
    group by 1
    )

    select b.TRADE_GROUP_DESC, d.Btlr_Org_LVL_C_Desc, coalesce(sum(f.Volume), 0) as total_volume, count(distinct f.sale_id) as sales_count
    from ab_inbev.silver.dim_channels b
    cross join dim_reg d
    left join ab_inbev.silver.fato_sales f on f.Btlr_Org_LVL_C_Desc = d.Btlr_Org_LVL_C_Desc and f.TRADE_CHNL_DESC = b.TRADE_CHNL_DESC
    group by 1,2
""")

display(df)

#escrita
df.write.mode("overwrite").saveAsTable("ab_inbev.gold.tb_volume_regiao_tradeGroup")

In [0]:
df = spark.sql("""
with dim_reg as (
  select Btlr_Org_LVL_C_Desc
  from ab_inbev.silver.fato_sales
  group by 1
)

select b.BRAND_NM, d.Btlr_Org_LVL_C_Desc, coalesce(sum(f.Volume), 0) as total_volume, count(distinct f.sale_id) as sales_count
from ab_inbev.silver.dim_brand b
cross join dim_reg d
left join ab_inbev.silver.fato_sales f on f.Btlr_Org_LVL_C_Desc = d.Btlr_Org_LVL_C_Desc and f.CE_BRAND_FLVR = b.CE_BRAND_FLVR
group by 1,2
""")

display(df)

#escrita
df.write.mode("overwrite").saveAsTable("ab_inbev.gold.tb_volume_regiao_brand")

### O que poderia ser melhorado nesse processo

- Mais testes de integracao e data quality
- Criar outras visoes importantes para o negocio
- Criar um pipeline para dar merge com novos dados que estejam no blob container ou em outra fonte
- Monitoria e logs
- Schedular pipelines via workflows
- Documentacao e governanca de dados (ex: colocar descricoes de colunas no catalogo do databricks)
- Padronizacao em nomes de colunas, tabelas e variaveis